<a href="https://colab.research.google.com/github/sskamalapurkar/Forage_British_Airways/blob/main/Forage_BA_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install beautifulsoup4

In [3]:
airline_url = "https://www.airlinequality.com/airline-reviews/british-airways/?sortby=post_date%3ADesc&pagesize=100"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36"
}

In [4]:
# Import required libraries
from bs4 import BeautifulSoup
import requests
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import unicodedata

In [5]:
html = requests.get(airline_url, headers=headers).text

In [6]:
bs = BeautifulSoup(html, "html.parser")

In [7]:
container = bs.find("article", {"class": "comp comp_reviews-airline querylist position-content"})

In [8]:
columns = ["Aircraft","Type Of Traveller","Seat Type","Route","Date Flown","Seat Comfort","Cabin Staff Service",
           "Food & Beverages","Ground Service","Inflight Entertainment","Wifi & Connectivity","Value For Money",
           "Recommended"]

df_columns = ["Overall_Rating","Review_Title","Review Date","Verified","Review","Aircraft",
              "Type Of Traveller","Seat Type","Route","Date Flown","Seat Comfort","Cabin Staff Service",
              "Food & Beverages","Ground Service","Inflight Entertainment","Wifi & Connectivity","Value For Money",
              "Recommended"]

In [9]:
reviews = []
# Check if the container exists (i.e., the review page is valid)
if container:
    # Loop through each review article in the container and extract relevant details
    for items in container.find_all("article"):
        verified = False
        # Extract the overall rating from the review
        rating = items.find("div", {"class": "rating-10"})
        if rating:
            rating = rating.text.strip()[:1]
        else:
            rating = None
        # Extract the review title from the review
        title = items.find("h2")
        if title:
            title = title.text
        else:
            title = None
        # Extract the review date from the review
        time = items.find("h3").find("time")
        if time:
            time = time.text
        else:
            time = None
        # Extract the review content from the review
        text = items.find("div", {"class": "text_content"})
        if text:
            text = text.text
            text = text.split("|")
            if len(text) == 1:
                review = text[0]
            else:
                if text[0] == '✅ Trip Verified ':
                    verified = True
                review = text[1]
        else:
            review = None
        # Extract ratings for specific categories from the review
        table = items.find("table")
        tab = [None] * 13
        if table:
            for item in table.find_all("tr"):
                i = 0
                for td in item.find_all("td"):
                    if i == 0:
                        condition = td.text
                        # Finding index of the given condition in the 'columns' list
                        ind = columns.index(condition)
                        i = 1
                    else:
                        # Checking if it's a rating or not
                        if td.find("span") is None:
                            value = td.text
                            tab[ind] = value
                        # In case it's a rating, counting stars filled (i.e., stars given)
                        else:
                            value = 0
                            for star in td.find_all("span", {"class": "star fill"}):
                                value += 1
                            tab[ind] = value

        # Store the extracted review data in a list named 'data'
        data = [rating, title, time, verified, review] + tab
        reviews.append(data)

# The 'reviews' list now contains all the extracted review data for different airlines.


In [10]:
#creating reviews dataframe
df = pd.DataFrame(reviews, columns=df_columns)

In [13]:
df.head(5)

,Overall_Rating,Review_Title,Review Date,Verified,Review,Aircraft,Type Of Traveller,Seat Type,Route,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Ground Service,Inflight Entertainment,Wifi & Connectivity,Value For Money,Recommended
0,1,"""an excellent experience""",2nd March 2024,True,I upgraded at check in to Club Europe seat 1...,A319,Solo Leisure,Business Class,Heathrow to Oslo,February 2024,4.0,5.0,5.0,5.0,NaN,NaN,5,yes
1,1,"""they refused to fly me back!",1st March 2024,True,"I bought a return trip with BA, through Webj...",None,Solo Leisure,Economy Class,Sydney to Porto via Singapore / London,June 2023,NaN,NaN,NaN,NaN,NaN,NaN,1,no
2,3,"""Poor from start to finish""",29th February 2024,True,Poor from start to finish. Six months after ...,A320,Couple Leisure,Economy Class,London Heathrow to Lisbon,February 2024,1.0,3.0,1.0,2.0,NaN,NaN,3,no
3,2,“customer service non existent”,29th February 2024,True,Communication and customer service non existe...,A320,Solo Leisure,Premium Economy,Madrid to London via Buenos Aires,February 2024,1.0,3.0,NaN,1.0,NaN,NaN,1,no
4,1,"""no better than the worst of the low cost""",23rd February 2024,True,That was supposed to be my flight but it was...,None,Couple Leisure,Economy Class,Pisa to London Heathrow,September 2023,NaN,NaN,NaN,NaN,NaN,NaN,1,no


In [12]:
df.isnull().sum()

Overall_Rating             0
Review_Title               0
Review Date                0
Verified                   0
Review                     0
Aircraft                  41
Type Of Traveller          0
Seat Type                  0
Route                      0
Date Flown                 0
Seat Comfort               5
Cabin Staff Service        6
Food & Beverages          19
Ground Service             2
Inflight Entertainment    47
Wifi & Connectivity       62
Value For Money            0
Recommended                0
dtype: int64